In [ ]:
import atdata
from dataclasses import dataclass
import atdata._local_dev as al

In [2]:
import redis

In [3]:
pool = redis.ConnectionPool(host='localhost', port=6379, db=0)
r = redis.Redis(connection_pool=pool)

# Check specific keys
r.keys('*')

[b':atdata.local.IndexEntry:01KB1CEARQXTTZDP1A8211645V',
 b':atdata.local.IndexEntry:01KB1D7R2SG9EDX5EHEFEC8R2N',
 b':atdata.local.IndexEntry:01KB1DDAW9HQ99XFF8FPFQ5AVT',
 b':atdata.local.IndexEntry:01KB1DMKTFGHY0ZH3VTNJ1H9XG',
 b':atdata.local.IndexEntry:index:hash',
 b':atdata.local.IndexEntry:01KB1CEHX24E9VDGG23DSMDZAF']

In [4]:
local_index = al.Index()

In [5]:
@dataclass
class TestSample( atdata.PackableSample ):
    identifier: int
    name: str

dataset = atdata.Dataset[TestSample]( '/data/test.tar' )

In [6]:
local_index.add( dataset )

IndexEntry(pk='01KB1DTP0QJZGV49J1339T9256', wds_url='/data/test.tar', sample_schema=SampleSchema(pk='01KB1DTP0PQ4XKV2N5HHQC3GWK', identifier='74deedc1-fbbb-4b11-9f99-cd73d3257983', json_schema={'$id': 'https://schema.dev/fake-schema.schema.json', '$schema': 'http://json-schema.org/draft-07/schema#', 'title': 'fake-schema', 'description': 'Blue Blah', 'type': 'object', 'properties': {'property_a': {'default': 5, 'type': 'integer'}, 'property_b': {'type': 'string'}}}), metadata_schema=None)

In [7]:
x: al.IndexEntry = local_index.list()[0]

In [8]:
x.sample_schema.json_schema

{'$id': 'https://schema.dev/fake-schema.schema.json',
 '$schema': 'http://json-schema.org/draft-07/schema#',
 'title': 'fake-schema',
 'description': 'Blue Blah',
 'type': 'object',
 'properties': {'property_a': {'default': 5, 'type': 'integer'},
  'property_b': {'type': 'string'}}}

In [46]:
def _test_init( self, a: int, b: str ):
    self.property_a = a
    self.property_b = b

TestType = type( 'TestType', (object,), {
    '__init__': _test_init,
    'property_a': 4,
    '__annotations__': {
        'property_a': int,
        'property_b': str,
    }
} )

In [ ]:
TestType.property_a

In [49]:
x = TestType( 4, 'shark' )

In [50]:
x.property_a

4

In [9]:
from schemamodels import SchemaModelFactory

factory = SchemaModelFactory()
factory.register( x.sample_schema.json_schema )

True

In [31]:
from schemamodels.dynamic import FakeSchema

@dataclass( frozen = False )
class FakeSchemaPackable( atdata.PackableSample, FakeSchema ):
    blah = 'hello'

TypeError: cannot inherit non-frozen dataclass from a frozen one

In [21]:
y = FakeSchema( property_a = 4, property_b = 'test' )

In [26]:
y.property_a

4

In [28]:
x = FakeSchemaPackable( property_a = 4, property_b = 'test' )

In [17]:
x.property_a

4

In [15]:
FakeSchemaPackable.from_bytes( x.as_wds['msgpack'] )

FakeSchemaPackable(property_b='', property_a=5)

In [18]:
xw = x.as_wds

In [19]:
xw

{'__key__': '36e2f3c0-cb2e-11f0-8000-000000000000', 'msgpack': b'\x80'}